In [3]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/campusx-official/jupyter-masterclass/main/tweet_emotions.csv').drop(columns=['tweet_id'])
df.head()

,sentiment,content
0,empty,@tiffanylue i know i was listenin to bad habi...
1,sadness,Layin n bed with a headache ughhhh...waitin o...
2,sadness,Funeral ceremony...gloomy friday...
3,enthusiasm,wants to hang out with friends SOON!
4,neutral,@dannycastillo We want to trade with someone w...


In [5]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise




In [6]:
df = normalize_text(df)
df.head()

,sentiment,content
0,empty,tiffanylue know listenin bad habit earlier sta...
1,sadness,layin n bed headache ughhhh waitin call
2,sadness,funeral ceremony gloomy friday
3,enthusiasm,want hang friend soon
4,neutral,dannycastillo want trade someone houston ticke...


In [7]:
df['sentiment'].value_counts()

sentiment
neutral       8638
worry         8459
happiness     5209
sadness       5165
love          3842
surprise      2187
fun           1776
relief        1526
hate          1323
empty          827
enthusiasm     759
boredom        179
anger          110
Name: count, dtype: int64

In [8]:
x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]


In [9]:
df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})
df.head()


C:\Users\senor\AppData\Local\Temp\ipykernel_27072\3057201912.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


,sentiment,content
1,0,layin n bed headache ughhhh waitin call
2,0,funeral ceremony gloomy friday
6,0,sleep im not thinking old friend want married ...
8,0,charviray charlene love miss
9,0,kelcouch sorry least friday


In [10]:
vectorizer = CountVectorizer(max_features=1000)
X = vectorizer.fit_transform(df['content'])
y = df['sentiment']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
# experiment tracking with mlflow and dagshub
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/senorhimanshu/mlops-mini-project.mlflow')
dagshub.init(repo_owner='senorhimanshu', repo_name='mlops-mini-project', mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")


Accessing as senorhimanshu

Initialized MLflow to track repo "senorhimanshu/mlops-mini-project"

Repository senorhimanshu/mlops-mini-project initialized!

2025/09/29 00:00:31 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/a720756140cb41a19db0f7485b5e9dce', creation_time=1759084232138, experiment_id='0', last_update_time=1759084232138, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [12]:
with mlflow.start_run():
    
    # Log preprocessing parameters
    mlflow.log_param("vectorizer", "Bag of Words")
    mlflow.log_param("num_features", 1000)
    mlflow.log_param("test_size", 0.2)
    
    # Model building and training
    model = LogisticRegression()
    model.fit(X_train, y_train)
    
    # Log model parameters
    mlflow.log_param("model", "Logistic Regression")
    
    # Model evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    # Log evaluation metrics
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1_score", f1)
    
    # Log model
    mlflow.sklearn.log_model(model, "model")

    # Save and log the notebook
    import os
    notebook_path = "exp1_baseline_model.ipynb"
    os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
    mlflow.log_artifact(notebook_path)
    
    # Print the results for verification
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")

Accuracy: 0.7773493975903615
Precision: 0.7692307692307693
Recall: 0.7783251231527094
F1 Score: 0.7737512242899118


In [12]:
## TEST

from sklearn.model_selection import GridSearchCV


# vectorizer = CountVectorizer()
# X = vectorizer.fit_transform(df['content'])
# y = df['sentiment']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set the experiment name
# mlflow.set_experiment("LoR Hyperparameter Tuning")

# Define hyperparameter grid for Logistic Regression
param_grid = {
    'C': [0.1, 1, 10],
    'penalty': ['l1', 'l2'],
    'solver': ['liblinear']
}

# Start the parent run for hyperparameter tuning
# with mlflow.start_run():

# Perform grid search
grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring='f1', n_jobs=-1)
grid_search.fit(X_train, y_train)


,estimator,LogisticRegression()
,param_grid,"{'C': [0.1, 1, ...], 'penalty': ['l1', 'l2'], 'solver': ['liblinear']}"
,scoring,'f1'
,n_jobs,-1
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,penalty,'l1'


In [13]:
grid_search.cv_results_

{'mean_fit_time': array([0.01548395, 0.01842289, 0.01782312, 0.03034806, 0.02261949,
        0.05767436]),
 'std_fit_time': array([0.00355155, 0.00317379, 0.01002534, 0.00752509, 0.00408995,
        0.02097675]),
 'mean_score_time': array([0.00726004, 0.00516229, 0.00411496, 0.00651889, 0.00609679,
        0.00554967]),
 'std_score_time': array([0.00495846, 0.00242468, 0.00380228, 0.00150833, 0.00113783,
        0.00231246]),
 'param_C': masked_array(data=[0.1, 0.1, 1.0, 1.0, 10.0, 10.0],
              mask=[False, False, False, False, False, False],
        fill_value=1e+20),
 'param_penalty': masked_array(data=['l1', 'l2', 'l1', 'l2', 'l1', 'l2'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_solver': masked_array(data=['liblinear', 'liblinear', 'liblinear', 'liblinear',
                    'liblinear', 'liblinear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
     

In [14]:
grid_search.best_params_

{'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}

In [15]:
grid_search.best_score_

0.7815448291429231

In [16]:
for params, mean_score, std_score in zip(grid_search.cv_results_['params'], grid_search.cv_results_['mean_test_score'], grid_search.cv_results_['std_test_score']):
    model = LogisticRegression(**params)
    model.fit(X_train, y_train)
    
    # Model evaluation
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    
    
    # Print the results for verification
    print(f"Mean CV Score: {mean_score}, Std CV Score: {std_score}")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1}")


Mean CV Score: 0.7051810430308207, Std CV Score: 0.014168003094990942
Accuracy: 0.7397590361445783
Precision: 0.7752027809965237
Recall: 0.6591133004926109
F1 Score: 0.7124600638977636
Mean CV Score: 0.7761091257882283, Std CV Score: 0.011020318487509659
Accuracy: 0.7812048192771084
Precision: 0.7741935483870968
Recall: 0.780295566502463
F1 Score: 0.7772325809617272
Mean CV Score: 0.7815448291429231, Std CV Score: 0.009831673653088673
Accuracy: 0.7840963855421687
Precision: 0.7793103448275862
Recall: 0.7793103448275862
F1 Score: 0.7793103448275862
Mean CV Score: 0.7763695526322832, Std CV Score: 0.010784720469479515
Accuracy: 0.7739759036144578
Precision: 0.7650485436893204
Recall: 0.7763546798029557
F1 Score: 0.7706601466992665
Mean CV Score: 0.7677881089671399, Std CV Score: 0.013319214031327352
Accuracy: 0.7677108433734939
Precision: 0.7569913211186113
Recall: 0.7733990147783252
F1 Score: 0.7651072124756335
Mean CV Score: 0.7699334741103928, Std CV Score: 0.012604901225284654
Accura

In [17]:
# Log the best run details in the parent run
best_params = grid_search.best_params_
best_score = grid_search.best_score_


print(f"Best Params: {best_params}")
print(f"Best F1 Score: {best_score}")

Best Params: {'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}
Best F1 Score: 0.7815448291429231


In [18]:
scoring = {
    'f1': 'f1',
    'accuracy': 'accuracy'
}

grid_search = GridSearchCV(LogisticRegression(), param_grid, cv=5, scoring=scoring, refit='f1', n_jobs=-1)
grid_search.fit(X_train, y_train)




,estimator,LogisticRegression()
,param_grid,"{'C': [0.1, 1, ...], 'penalty': ['l1', 'l2'], 'solver': ['liblinear']}"
,scoring,"{'accuracy': 'accuracy', 'f1': 'f1'}"
,n_jobs,-1
,refit,'f1'
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,penalty,'l1'


In [19]:
grid_search.cv_results_

{'mean_fit_time': array([0.01252093, 0.01800046, 0.01869459, 0.0347199 , 0.02453485,
        0.05800986]),
 'std_fit_time': array([0.00352898, 0.00508749, 0.00557516, 0.00286495, 0.00106973,
        0.00656354]),
 'mean_score_time': array([0.00788469, 0.00770607, 0.00861068, 0.00956025, 0.00854111,
        0.00887213]),
 'std_score_time': array([0.00143007, 0.0011438 , 0.00347923, 0.00095315, 0.00083982,
        0.00090702]),
 'param_C': masked_array(data=[0.1, 0.1, 1.0, 1.0, 10.0, 10.0],
              mask=[False, False, False, False, False, False],
        fill_value=1e+20),
 'param_penalty': masked_array(data=['l1', 'l2', 'l1', 'l2', 'l1', 'l2'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_solver': masked_array(data=['liblinear', 'liblinear', 'liblinear', 'liblinear',
                    'liblinear', 'liblinear'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
     

In [22]:
grid_search.cv_results_['mean_test_accuracy']

array([0.72996906, 0.77708243, 0.78009586, 0.77515596, 0.76672157,
       0.76852902])

In [23]:
grid_search.cv_results_['mean_test_accuracy'][grid_search.best_index_]

0.7800958626549599

In [ ]:
# Best accuracy from CV
best_accuracy_cv = grid_search.cv_results_['mean_test_accuracy'][grid_search.best_index_]
mlflow.log_metric("best_accuracy_cv", best_accuracy_cv)

In [5]:
# import json
# import mlflow
# import logging
# import os
# import dagshub

# # Set up DagsHub credentials for MLflow tracking
# dagshub_token = os.getenv("DAGSHUB_PAT")
# if not dagshub_token:
#     raise EnvironmentError("DAGSHUB_PAT environment variable is not set")

# os.environ["MLFLOW_TRACKING_USERNAME"] = dagshub_token
# os.environ["MLFLOW_TRACKING_PASSWORD"] = dagshub_token

In [7]:
import pickle

In [ ]:
@classmethod
def setUpClass(cls):
    # Set up DagsHub credentials for MLflow tracking
    dagshub_token = os.getenv("DAGSHUB_PAT")
    if not dagshub_token:
        raise EnvironmentError("DAGSHUB_PAT environment variable is not set")

    os.environ["MLFLOW_TRACKING_USERNAME"] = dagshub_token
    os.environ["MLFLOW_TRACKING_PASSWORD"] = dagshub_token

    dagshub_url = "https://dagshub.com"
    repo_owner = "senorhimanshu"
    repo_name = "mlops-mini-project"

    # Set up MLflow tracking URI
    mlflow.set_tracking_uri(f'{dagshub_url}/{repo_owner}/{repo_name}.mlflow')

    # Load the new model from MLflow model registry
    cls.new_model_name = "my_model"
    cls.new_model_version = cls.get_latest_model_version(cls.new_model_name)
    cls.new_model_uri = f'models:/{cls.new_model_name}/{cls.new_model_version}'
    cls.new_model = mlflow.pyfunc.load_model(cls.new_model_uri)

    # Load the vectorizer
    cls.vectorizer = pickle.load(open('models/vectorizer.pkl', 'rb'))

    # Load holdout test data
    cls.holdout_data = pd.read_csv('data/processed/test_bow.csv')

In [ ]:
def test_model_signature(self):
    # Create a dummy input for the model based on expected input shape
    input_text = "hi how are you"
    input_data = self.vectorizer.transform([input_text])
    input_df = pd.DataFrame(input_data.toarray(), columns=[str(i) for i in range(input_data.shape[1])])

    # Predict using the new model to verify the input and output shapes
    prediction = self.new_model.predict(input_df)

    # Verify the input shape
    self.assertEqual(input_df.shape[1], len(self.vectorizer.get_feature_names_out()))

    # Verify the output shape (assuming binary classification with a single output)
    self.assertEqual(len(prediction), input_df.shape[0])
    self.assertEqual(len(prediction.shape), 1)  # Assuming a single output column for binary classification


In [ ]:
vectorizer = pickle.load(open('../models/vectorizer.pkl', 'rb'))

In [14]:
input_text = "hi how are you"
input_data = vectorizer.transform([input_text])

In [15]:
input_data

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 4 stored elements and shape (1, 5000)>

In [22]:
input_df = pd.DataFrame(input_data.toarray(), columns=[str(i) for i in range(input_data.shape[1])])

In [23]:
input_df

,0,1,2,3,4,5,6,7,8,9,...,4990,4991,4992,4993,4994,4995,4996,4997,4998,4999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
input_data.toarray()

array([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [24]:
columns = [str(i) for i in range(input_data.shape[1])]

In [25]:
new_model = pickle.load(open("../models/model.pkl", "rb"))

In [26]:
prediction = new_model.predict(input_df)

c:\Users\senor\anaconda3\envs\mlops_ci\lib\site-packages\sklearn\utils\validation.py:2742: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [ ]:
self.assertEqual(input_df.shape[1], len(self.vectorizer.get_feature_names_out()))

In [27]:
len(vectorizer.get_feature_names_out())

5000

In [28]:
len(prediction)

1

In [29]:
input_df.shape[0]

1

In [30]:
len(prediction.shape)

1

In [3]:
import dagshub
print(dagshub.__version__)

0.6.3
